# T1057 - Process Discovery
Adversaries may attempt to get information about running processes on a system. Information obtained could be used to gain an understanding of common software/applications running on systems within the network. Adversaries may use the information from [Process Discovery](https://attack.mitre.org/techniques/T1057) during automated discovery to shape follow-on behaviors, including whether or not the adversary fully infects the target and/or attempts specific actions.

In Windows environments, adversaries could obtain details on running processes using the [Tasklist](https://attack.mitre.org/software/S0057) utility via [cmd](https://attack.mitre.org/software/S0106) or <code>Get-Process</code> via [PowerShell](https://attack.mitre.org/techniques/T1059/001). Information about processes can also be extracted from the output of [Native API](https://attack.mitre.org/techniques/T1106) calls such as <code>CreateToolhelp32Snapshot</code>. In Mac and Linux, this is accomplished with the <code>ps</code> command. Adversaries may also opt to enumerate processes via /proc.

## Atomic Tests

### Atomic Test #1 - Process Discovery - ps
Utilize ps to identify processes.

Upon successful execution, sh will execute ps and output to /tmp/loot.txt.

**Supported Platforms:** macos, linux
#### Attack Commands: Run with `sh`
```sh
ps >> /tmp/loot.txt
ps aux >> /tmp/loot.txt
```

In [ ]:
Invoke-AtomicTest T1057 -TestNumbers 1

### Atomic Test #2 - Process Discovery - tasklist
Utilize tasklist to identify processes.

Upon successful execution, cmd.exe will execute tasklist.exe to list processes. Output will be via stdout. 

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
tasklist
```

In [ ]:
Invoke-AtomicTest T1057 -TestNumbers 2

### Atomic Test #3 - Process Discovery - Get-Process
Utilize Get-Process PowerShell cmdlet to identify processes.

Upon successful execution, powershell.exe will execute Get-Process to list processes. Output will be via stdout. 

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
Get-Process
```

In [ ]:
Invoke-AtomicTest T1057 -TestNumbers 3

### Atomic Test #4 - Process Discovery - get-wmiObject
Utilize get-wmiObject PowerShell cmdlet to identify processes.

Upon successful execution, powershell.exe will execute get-wmiObject to list processes. Output will be via stdout. 

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
get-wmiObject -class Win32_Process
```

In [ ]:
Invoke-AtomicTest T1057 -TestNumbers 4

### Atomic Test #5 - Process Discovery - wmic process
Utilize windows management instrumentation to identify processes.

Upon successful execution, WMIC will execute process to list processes. Output will be via stdout. 

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic process get /format:list
```

In [ ]:
Invoke-AtomicTest T1057 -TestNumbers 5

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Normal, benign system and network events that look like process discovery may be uncommon, depending on the environment and how they are used. Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).

## Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
A defender can modify commands such that the true list of running processes is not revealed, hiding necessary active defense processes from the adversary.
#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.